In [1]:
import time
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.types import DoubleType

# minhash

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, ArrayType, IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinHashLSH
from pyspark.sql.functions import udf,col
from pyspark.sql.types import LongType, FloatType

import random

config = SparkConf().setAppName("LSH").setMaster("local[8]")
sc = SparkContext.getOrCreate(config)
spark = SparkSession(sc)

In [3]:
import re

SHINGLE_LENGTH = 3
PRIME = 2147483647
SIMILARITY_THRESHOLD = 0.001
BAND_SIMILARITY_THRESHOLD = 0.001
NUMBER_OF_BANDS = 10
NUMBER_OF_ROWS = 5
NUMBER_OF_HASHFUNCTIONS = 20

class minLSH:
    def __init__(self, numHashTables=5, shingleSize=5, inputCol="features", outputCol="hashes"):
        self.numHashTables = numHashTables
        self.shingleSize = shingleSize
        self.inputCol = inputCol
        self.outputCol = outputCol
        self.hashFunctions = self.getHashFunctions(self.numHashTables)

    def getHashFunctions(self, number):
        result = []
        rand = random.Random()
        for _ in range(number):
            a = rand.randint(0, PRIME-1)
            b = rand.randint(0, PRIME-1)
            result.append(lambda x, a=a, b=b: ((a * x + b) % PRIME))
        return result
    
    def preprocessDocument(self, document):
        return document.strip().lower().replace("[^\\w\\s]", "").replace("\\s+", " ") if isinstance(document, str) else document

    def shingle(self, document):
        resultingList = []
        i = 0
        while i + self.shingleSize < len(document):
            resultingList.append(hash(document[i:i+self.shingleSize]))
            i += 1
        return resultingList

    def minHash(self, listOfShingles):
        result = []
        for ind in range(len(self.hashFunctions)):
            minVal = float("inf")
            for shingle in listOfShingles:
                hashResult = self.hashFunctions[ind](shingle)
                if hashResult < minVal:
                    minVal = hashResult
            result.append(minVal)
        return result
    
    def signatureToHashedBandsOfRows(self, signature, numberOfBands, numberOfRowsInBand):
        if len(signature) != numberOfBands * numberOfRowsInBand:
            raise Exception("Wrong arguments number of bands times number of rows should equal length of signature")
        i = 0
        bands = []
        while i + numberOfRowsInBand <= len(signature):
            bands.append(signature[i:i+numberOfRowsInBand])
            i += numberOfRowsInBand
        return [hash(tuple(band)) for band in bands]

    def fit(self, dataframe):
        return self

    def transform(self, dataframe):
        preprocessDocument_udf = udf(self.preprocessDocument, StringType())
        df = dataframe.withColumn("document", preprocessDocument_udf(self.inputCol))
        # 分词
        shingle_udf = udf(self.shingle, ArrayType(LongType()))
        df = df.withColumn("shingles", shingle_udf("document")).drop("document")
        # MinHash
        minHash_udf = udf(lambda x: self.minHash(x), ArrayType(LongType()))
        df = df.withColumn("minHash", minHash_udf("shingles")).drop("shingles")
        return df

    def approxSimilarityJoin(self, dataframeA, dataframeB, threshold, joinID="id"):
        # 相似度 udf
        jaccard_similarity = udf(lambda x, y: len(
                    set(x).intersection(set(y))) / len(set(x).union(set(y))), FloatType())

        # 相似度
        similarities = dataframeA.alias("left").join(dataframeB.alias("right"), on=joinID, how="inner") \
            .withColumn("similarity", jaccard_similarity(col("left.minHash"), col("right.minHash")))  \
            .filter(col("similarity") >= threshold)

        return similarities

In [4]:
dfA = spark.read.csv("quora/train.csv", header=True).select("id", "question1", "is_duplicate").withColumnRenamed("question1", "sentence")
dfB = spark.read.csv("quora/train.csv", header=True).select("id", "question2", "is_duplicate").withColumnRenamed("question2", "sentence")

dfA = dfA.where(dfA.is_duplicate.isin([0,1]))
dfA = dfA.where("sentence is not null")
dfB = dfB.where(dfB.is_duplicate.isin([0,1]))
dfB = dfB.where("sentence is not null")

dfA.show(5)
dfB.show(5)

+---+--------------------+------------+
| id|            sentence|is_duplicate|
+---+--------------------+------------+
|  0|What is the step ...|           0|
|  1|What is the story...|           0|
|  2|How can I increas...|           0|
|  3|Why am I mentally...|           0|
|  4|Which one dissolv...|           0|
+---+--------------------+------------+
only showing top 5 rows

+---+--------------------+------------+
| id|            sentence|is_duplicate|
+---+--------------------+------------+
|  0|What is the step ...|           0|
|  1|What would happen...|           0|
|  2|How can Internet ...|           0|
|  3|Find the remainde...|           0|
|  4|Which fish would ...|           0|
+---+--------------------+------------+
only showing top 5 rows



In [5]:
# num of partition 对时间的影响

for numPar in [5, 10, 20, 50]:
    A = dfA.repartition(numPar)
    B = dfB.repartition(numPar)
    
    start_time = time.time()
    
    model = minLSH(numHashTables=10, shingleSize=3, inputCol="sentence", outputCol="hashes")
    A = model.transform(A)
    B = model.transform(B)
    
    result = model.approxSimilarityJoin(A, B, 0, joinID="id")
    
    end_time = time.time()
    run_time = end_time - start_time
    
    print("Num of partition:", numPar, "; Time:", run_time)

Num of partition: 5 ; Time: 0.23078298568725586
Num of partition: 10 ; Time: 0.19745111465454102
Num of partition: 20 ; Time: 0.15598821640014648
Num of partition: 50 ; Time: 0.14315271377563477


In [6]:
# AUC

tmp = result.select("id", "right.is_duplicate", "similarity")
tmp = tmp.withColumn("is_duplicate", tmp["is_duplicate"].cast(DoubleType()))
metrics = BinaryClassificationMetrics(tmp.select("is_duplicate", "similarity").rdd.map(tuple))
metrics.areaUnderROC

/home/jy/spark-3.3.1-bin-hadoop2/python/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


0.5962023321616184

# simhash

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, FloatType, ArrayType
from pyspark.ml.feature import HashingTF

spark = SparkSession.builder.appName("SimHashLSH").getOrCreate()

def preprocessDocument(document):
    return document.strip().lower().replace("[^\\w\\s]", "").replace("\\s+", " ")

# 定义 SimHash 函数
def simhash(document):
    document = preprocessDocument(document)
    # 将文档拆分为单词
    words = document.split(" ")

    # 计算每个单词的 SimHash 值
    hashes = []
    for word in words:
        word_hash = hash(word)
        # 使用 64 位 SimHash 值，将每个单词的哈希值转换为二进制表示，并填充到 64 位
        binary_hash = format(word_hash, "064b")
        # 将二进制表示的哈希值转换为 DenseVector，每个元素值为 -1 或 1
        vector = [1 if b == "1" else -1 for b in binary_hash]
        hashes.append(vector)
    # 将所有单词的 SimHash 值合并到一个数组中
    simhash_value = [0] * 64
    for v in hashes:
        for i in range(64):
            simhash_value[i] += v[i]
    simhash_value = [1 if x > 0 else 0 for x in simhash_value]
    return simhash_value

# 将 SimHash 函数注册为 UDF
simhash_udf = udf(simhash, ArrayType(IntegerType()))

# 定义 Hamming Distance 函数
def hamming_distance(simhash1, simhash2):
    # 计算两个 SimHash 值的 Hamming 距离
    distance = sum([1 for i in range(64) if simhash1[i] != simhash2[i]])
    return distance

# 将 Hamming Distance 函数注册为 UDF
hamming_distance_udf = udf(hamming_distance, IntegerType())

# 定义相似度计算函数 similarity = hamming_distance / 64
def similarity(simhash1, simhash2):
    length = len(simhash1)
    cnt = sum([1 for i in range(length) if simhash1[i] == simhash2[i]])
    return cnt / length

# 将相似度计算函数注册为 UDF
similarity_udf = udf(similarity, FloatType())

23/04/23 09:25:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
dfA = spark.read.csv("quora/train.csv", header=True).select("id", "question1", "is_duplicate")
dfB = spark.read.csv("quora/train.csv", header=True).select("id", "question2", "is_duplicate")

dfA = dfA.where(dfA.is_duplicate.isin([0,1]))
dfA = dfA.where("question1 is not null")
dfB = dfB.where(dfB.is_duplicate.isin([0,1]))
dfB = dfB.where("question2 is not null")

dfA.show(5)
dfB.show(5)

+---+--------------------+------------+
| id|           question1|is_duplicate|
+---+--------------------+------------+
|  0|What is the step ...|           0|
|  1|What is the story...|           0|
|  2|How can I increas...|           0|
|  3|Why am I mentally...|           0|
|  4|Which one dissolv...|           0|
+---+--------------------+------------+
only showing top 5 rows

+---+--------------------+------------+
| id|           question2|is_duplicate|
+---+--------------------+------------+
|  0|What is the step ...|           0|
|  1|What would happen...|           0|
|  2|How can Internet ...|           0|
|  3|Find the remainde...|           0|
|  4|Which fish would ...|           0|
+---+--------------------+------------+
only showing top 5 rows



In [13]:
for numPar in [5, 10, 20, 50]:
    
    # 改成多个分区数，对比运行时间
    dfA = dfA.repartition(numPar)
    dfB = dfB.repartition(numPar)
    
    start_time = time.time()
    
    # 对文档进行 SimHash 计算
    dfA = dfA.withColumn("simhash_A", simhash_udf(dfA["question1"]))
    dfB = dfB.withColumn("simhash_B", simhash_udf(dfB["question2"]))

    # 计算文档之间的相似度
    similar_documents = dfA.join(dfB, 'id') \
        .select('id', 'question1', 'question2', 
                similarity_udf(col("simhash_A"), col("simhash_B")).alias("similarity"),
                hamming_distance_udf(col("simhash_A"), col("simhash_B")).alias("distance"))
    
    end_time = time.time()
    run_time = end_time - start_time
    
    print("Num of partition:", numPar, "; Time:", run_time)

Num of partition: 5 ; Time: 0.041646480560302734
Num of partition: 10 ; Time: 0.03614950180053711
Num of partition: 20 ; Time: 0.03628945350646973
Num of partition: 50 ; Time: 0.04783296585083008


In [14]:
result = similar_documents.select("id", "similarity").join(dfB.select("id", "is_duplicate"), "id")
result.show(5)

+------+----------+------------+
|    id|similarity|is_duplicate|
+------+----------+------------+
|100010|   0.84375|           0|
|100014|   0.78125|           1|
|100021|    0.5625|           0|
|100062|   0.59375|           0|
|100070|    0.5625|           0|
+------+----------+------------+
only showing top 5 rows



In [15]:
# AUC

tmp = result.withColumn("is_duplicate", result["is_duplicate"].cast(DoubleType()))
metrics = BinaryClassificationMetrics(tmp.select("is_duplicate", "similarity").rdd.map(tuple))
metrics.areaUnderROC

0.6652052088216065